In [1]:
import pandas as pd

In [2]:
learners = ['OLS'] #['RF', 'AML']
nuisances = ['y_hat', 'y0_hat', 'y1_hat', 'p_hat']
scores_l = ["T", "F", "U", "D"]

In [3]:
def calc_T(y1_hat, y0_hat):
    return y1_hat - y0_hat

In [4]:
def calc_F(y, D, p_hat):
    return y * (D - p_hat) / p_hat / (1-p_hat)

In [5]:
def calc_U(y, D, y_hat, p_hat):
    return (y - y_hat) / (D - p_hat)

In [6]:
def calc_D(y, D, y0_hat, y1_hat, p_hat):
    return y1_hat - y0_hat + D * (y - y1_hat) / p_hat - (1-D) * (y - y0_hat) / (1-p_hat)

In [8]:
for knob in range(9):
    for dataset in range(10):
        estimates = {learner:pd.DataFrame(index=range(47684), columns=nuisances) for learner in learners}
        scores = {learner:pd.DataFrame(index=range(47684), columns=scores_l) for learner in learners}
        df = pd.read_csv("C:/DISSERTATION/Data/dataset_"+str(knob)+str(dataset)+".csv", index_col=0)
        for learner in learners:
            for nuisance in nuisances:
                estimates[learner][nuisance] = pd.read_csv("C:/DISSERTATION/Results/"+learner+"/"+nuisance+"/dataset_"+str(knob)+str(dataset)+".csv", index_col=0)
            estimates[learner]['p_hat'] = estimates[learner]['p_hat'].apply(lambda x: max(0.01, x))
            scores[learner]["T"] = calc_T(estimates[learner]["y1_hat"], estimates[learner]["y0_hat"])
            scores[learner]["F"] = calc_F(df['y'], df['D'], estimates[learner]['p_hat'])
            scores[learner]["U"] = calc_U(df['y'], df['D'], estimates[learner]['y_hat'], estimates[learner]['p_hat'])
            scores[learner]["D"] = calc_D(df['y'], df['D'], estimates[learner]['y0_hat'], estimates[learner]['y1_hat'], estimates[learner]['p_hat'])
            scores[learner].to_csv("C:/DISSERTATION/Results/"+learner+"/scores/dataset_"+str(knob)+str(dataset)+".csv")